In [2]:
from dwave.system import DWaveSampler, EmbeddingComposite
from greedy import SteepestDescentComposite
import sys, os
sys.path.append("../")
from bqm_input_checker import InputChecker
from btor2bqm import BTor2BQM

# 32 bit

In [3]:
parser = BTor2BQM(15)
bqm_32, total_time_32, num_variables_32 = parser.parse_file(f"../../../qa_examples/32_u.btor2",
                          f"./32_u/",
                          with_init=True, modify_memory_sort=True)
print(total_time_32, num_variables_32)

started building ../../../qa_examples/32_u.btor2 for 15 timesteps
{'begin_datasegment': 17408, 'begin_heap': 18432, 'size_datasegment': 2, 'size_heap': 1, 'size_stack': 12, 'word_size': 32, 'address_step_size': 1, 'address_word_size': 30, 'begin_stack': 1073741824}
output dir:  ./32_u/
sort memory modified to be bitvector of size:  480
{1: ['1', 'sort', 'bitvec', '1'], 2: ['2', 'sort', 'bitvec', '32'], 4: ['4', 'sort', 'bitvec', '30'], 5: ['5', 'sort', 'bitvec', '480'], 10: ['10', 'zero', '1'], 11: ['11', 'one', '1'], 20: ['20', 'zero', '2'], 21: ['21', 'one', '2'], 22: ['22', 'constd', '2', '2'], 23: ['23', 'constd', '2', '3'], 24: ['24', 'constd', '2', '4'], 30: ['30', 'constd', '2', '69632'], 31: ['31', 'constd', '2', '69640'], 32: ['32', 'constd', '2', '73728'], 33: ['33', 'constd', '2', '73732'], 40: ['40', 'constd', '4', '17408'], 41: ['41', 'constd', '4', '17410'], 42: ['42', 'constd', '4', '18432'], 43: ['43', 'constd', '4', '18433'], 50: ['50', 'constd', '2', '4294967292'], 60

0.35 348


# 64 bit

In [4]:
parser64 = BTor2BQM(15)
bqm_64, total_time_64, num_variables_64 = parser64.parse_file(f"../../../qa_examples/64_u.btor2",
                          f"./64_u/",
                          with_init=True, modify_memory_sort=True)
print(total_time_64, num_variables_64)

started building ../../../qa_examples/64_u.btor2 for 15 timesteps
{'begin_datasegment': 8704, 'begin_heap': 9216, 'size_datasegment': 2, 'size_heap': 1, 'size_stack': 10, 'word_size': 64, 'address_step_size': 1, 'address_word_size': 29, 'begin_stack': 536870912}
output dir:  ./64_u/
sort memory modified to be bitvector of size:  832
{1: ['1', 'sort', 'bitvec', '1'], 2: ['2', 'sort', 'bitvec', '64'], 4: ['4', 'sort', 'bitvec', '29'], 5: ['5', 'sort', 'bitvec', '832'], 10: ['10', 'zero', '1'], 11: ['11', 'one', '1'], 20: ['20', 'zero', '2'], 21: ['21', 'one', '2'], 22: ['22', 'constd', '2', '2'], 23: ['23', 'constd', '2', '3'], 24: ['24', 'constd', '2', '4'], 25: ['25', 'constd', '2', '5'], 26: ['26', 'constd', '2', '6'], 27: ['27', 'constd', '2', '7'], 28: ['28', 'constd', '2', '8'], 30: ['30', 'constd', '2', '69632'], 31: ['31', 'constd', '2', '69648'], 32: ['32', 'constd', '2', '73728'], 33: ['33', 'constd', '2', '73736'], 40: ['40', 'constd', '4', '8704'], 41: ['41', 'constd', '4', '

0.66 398


# Minor Embeddings

In [5]:
import dwave_networkx as dnx

In [6]:
pegasus_graph = dnx.pegasus_graph(16)
len(pegasus_graph.nodes)

5640

In [7]:
chimera_graph = dnx.chimera_graph(16)
len(chimera_graph.nodes)

2048

In [8]:
import minorminer
import dimod
problem_graph_32 = dimod.to_networkx_graph(bqm_32)
problem_graph_64 = dimod.to_networkx_graph(bqm_64)

In [9]:
def get_count_embedding_physical_variables(embedding):
    result = 0
    for (key, ancillas) in embedding.items():
        result += len(ancillas)
    return result
def get_chain_lengths(embedding):
    result = []
    for (key, ancillas) in embedding.items():
        result.append(len(ancillas))
    return result

In [10]:
import time
import random
def get_statistics_embedding(target_graph, source_graph, num_runs, seed=1):
    random.seed(seed)
    times = []
    distribution_chain_lengths = []
    vars_in_qpu = []
    for _ in range(num_runs):
        embedding_seed = random.randint(0, 10000)
        t0 = time.perf_counter()
        embedding =  minorminer.find_embedding(source_graph, target_graph, random_seed=embedding_seed)
        t1 = time.perf_counter()
        times.append(round(t1-t0, 2))
        vars_in_qpu.append(get_count_embedding_physical_variables(embedding))
        distribution_chain_lengths.extend(get_chain_lengths(embedding))
    return times, distribution_chain_lengths, vars_in_qpu

In [11]:
import statistics
import pandas as pd
COL_FILE_NAME = 'file_name'
COL_WORDSIZE = 'wordsize'
COL_LOGIC_VARS = 'logic_vars'
COL_TOPOLOGY = 'topology'

COL_EMB_MAX_TIME = 'emb_max_time'
COL_EMB_MIN_TIME = 'emb_min_time'
COL_EMB_MEDIAN_TIME = 'emb_median_time'
COL_EMB_AVG_TIME = 'emb_avg_time'

COL_EMB_MAX_CHAIN = 'emb_max_chain'
COL_EMB_MIN_CHAIN = 'emb_min_chain'
COL_EMB_MEDIAN_CHAIN = 'emb_median_chain'
COL_EMB_AVG_CHAIN = 'emb_avg_chain'

COL_EMB_MAX_VARS = 'emb_max_vars'
COL_EMB_MIN_VARS = 'emb_min_vars'
COL_EMB_MEDIAN_VARS = 'emb_median_vars'
COL_EMB_AVG_VARS = 'emb_avg_vars'

columns = [COL_FILE_NAME, COL_WORDSIZE, COL_LOGIC_VARS, COL_EMB_MAX_TIME, COL_EMB_MIN_TIME, 
           COL_EMB_MEDIAN_TIME, COL_EMB_AVG_TIME, COL_EMB_MAX_CHAIN, 
           COL_EMB_MIN_CHAIN, COL_EMB_MEDIAN_CHAIN, COL_EMB_AVG_CHAIN,
          COL_EMB_MAX_VARS, COL_EMB_MIN_VARS, COL_EMB_MEDIAN_VARS, COL_EMB_AVG_VARS]
data_embedding = pd.DataFrame(columns=columns)
def add_embedding_statistics_row(filename, wordsize, topology, logic_vars, times, chains, vars_qpu):
    times.sort()
    chains.sort()
    vars_qpu.sort()
    
    row = {
        COL_FILE_NAME: filename,
        COL_WORDSIZE: wordsize,
        COL_LOGIC_VARS: logic_vars,
        COL_TOPOLOGY: topology,
        
        COL_EMB_MAX_TIME: max(times),
        COL_EMB_MIN_TIME: min(times),
        COL_EMB_MEDIAN_TIME: round(statistics.median(times),2),
        COL_EMB_AVG_TIME: round(sum(times)/len(times),2),
        
        COL_EMB_MAX_CHAIN: max(chains),
        COL_EMB_MIN_CHAIN: min(chains),
        COL_EMB_MEDIAN_CHAIN: round(statistics.median(chains),2),
        COL_EMB_AVG_CHAIN: round(sum(chains)/len(chains),2),
        
        COL_EMB_MAX_VARS: max(vars_qpu),
        COL_EMB_MIN_VARS: min(vars_qpu),
        COL_EMB_MEDIAN_VARS: round(statistics.median(vars_qpu),2),
        COL_EMB_AVG_VARS: round(sum(vars_qpu)/len(vars_qpu),2),
    }
    return data_embedding.append(row, ignore_index=True)
    

### Quimera Graph

In [12]:
chimera_times_32, chimera_chain_length_32, chimera_vars_qpu_32 = get_statistics_embedding(chimera_graph, problem_graph_32, 10)

In [13]:
data_embedding = add_embedding_statistics_row('u.btor2', 32, 'chimera', num_variables_32, chimera_times_32, chimera_chain_length_32, chimera_vars_qpu_32)

In [14]:
chimera_times_64, chimera_chain_length_64, chimera_vars_qpu_64 = get_statistics_embedding(chimera_graph, problem_graph_64, 10)

In [15]:
data_embedding = add_embedding_statistics_row('u.btor2', 64, 'chimera', num_variables_64, chimera_times_64, chimera_chain_length_64, chimera_vars_qpu_64)

In [16]:
data_embedding

,file_name,wordsize,logic_vars,emb_max_time,emb_min_time,emb_median_time,emb_avg_time,emb_max_chain,emb_min_chain,emb_median_chain,emb_avg_chain,emb_max_vars,emb_min_vars,emb_median_vars,emb_avg_vars,topology
0,u.btor2,32,348,23.20,0.87,5.41,8.29,31,1,2.0,3.18,1201,998,1110.0,1106.0,chimera
1,u.btor2,64,398,26.77,2.43,9.45,12.16,29,1,2.0,2.83,1203,1052,1126.0,1124.9,chimera


### Pegasus Graph

In [17]:
pegasus_times_32, pegasus_chain_length_32, pegasus_physical_vars_32 = get_statistics_embedding(pegasus_graph, problem_graph_32, 10)



In [18]:
data_embedding = add_embedding_statistics_row('u.btor2', 32, 'pegasus', num_variables_32, pegasus_times_32, pegasus_chain_length_32, pegasus_physical_vars_32)

In [19]:
pegasus_times_64, pegasus_chain_length_64, pegasus_physical_vars_64 = get_statistics_embedding(pegasus_graph, problem_graph_64, 10)



In [20]:
data_embedding = add_embedding_statistics_row('u.btor2', 64, 'pegasus', num_variables_64, pegasus_times_64, pegasus_chain_length_64, pegasus_physical_vars_64)

In [21]:
data_embedding

,file_name,wordsize,logic_vars,emb_max_time,emb_min_time,emb_median_time,emb_avg_time,emb_max_chain,emb_min_chain,emb_median_chain,emb_avg_chain,emb_max_vars,emb_min_vars,emb_median_vars,emb_avg_vars,topology
0,u.btor2,32,348,23.20,0.87,5.41,8.29,31,1,2.0,3.18,1201,998,1110.0,1106.0,chimera
1,u.btor2,64,398,26.77,2.43,9.45,12.16,29,1,2.0,2.83,1203,1052,1126.0,1124.9,chimera
2,u.btor2,32,348,4.44,1.87,3.08,3.12,11,1,1.0,1.69,628,555,584.5,588.6,pegasus
3,u.btor2,64,398,7.82,3.89,4.87,5.29,12,1,1.0,1.70,711,652,680.5,677.0,pegasus


In [22]:
data_embedding.to_csv('u_embedding.csv')

# Simulated Annealing

In [5]:
import neal
sampler = neal.SimulatedAnnealingSampler()
simulated_samples_32 = sampler.sample(bqm_32,num_reads=1000, seed=1).lowest()
print(len(simulated_samples_32.lowest()), simulated_samples_32.first.energy)

40 0.0


In [6]:
import neal
sampler = neal.SimulatedAnnealingSampler()
simulated_samples_64 = sampler.sample(bqm_64,num_reads=1000, seed=1).lowest()
print(len(simulated_samples_64.lowest()), simulated_samples_64.first.energy)

29 0.0


In [218]:
for sample in simulated_samples_32.lowest():
    input_dec = 0
    input_variables = [9376, 9377, 9378, 9379, 9380, 9381, 9382, 9383]
    for (index, input_variable) in enumerate(input_variables):
        input_dec += sample[input_variable] * (2**index)
    assert(input_dec > 0)
    
for sample in simulated_samples_64.lowest():
    input_dec = 0
    input_variables = [74187, 74188, 74189, 74190, 74191, 74192, 74193, 74194]
    for (index, input_variable) in enumerate(input_variables):
        input_dec += sample[input_variable] * (2**index)
    assert(input_dec > 0)

# Quantum Annealer

In [128]:
def get_max_chain_strength(bqm):
    return max(bqm.quadratic.values())

In [129]:
max_chain_strength_32 = get_max_chain_strength(bqm_32)
max_chain_strength_64 = get_max_chain_strength(bqm_64)
print(max_chain_strength_32)
print(max_chain_strength_64)

4.0
4.0


In [ ]:
chain_strengths_to_try = [1,2]
samples_numbers_to_try = [1000, 5000, 7000, 10000]

In [181]:
from dwave.system import DWaveSampler, EmbeddingComposite
from greedy import SteepestDescentComposite
DWAVE_2000 = "DW_2000Q_6"
ADVANTAGE = "Advantage_system4.1"

In [182]:
def sample_qpu(bqm, qpu_name, num_runs, chain_strength):
    random.seed(1)
    embedding_seed = random.randint(0, 10000)
    qpu = EmbeddingComposite(DWaveSampler(solver={"name": qpu_name}), embedding_parameters= {'random_seed':embedding_seed})
    sampler = SteepestDescentComposite(qpu)
    result = sampler.sample(bqm, num_reads=num_runs, chain_strength=chain_strength)
    return result

def get_energy_0_count(samples):
    answer = 0
    for sample in samples.lowest().record:
        energy = sample[1]
        occs = sample[2]
        if energy > 0:
            break
        answer += occs
    return answer

def get_good_answers_overall(samples, unicorn_files_path, input_variables):
    good_answers_overall = 0
    for (index, sample) in enumerate(samples):
        occs = samples.record[index][2]
        input_dec = 0
        for (index,input_variable) in enumerate(input_variables):
            input_dec += sample[input_variable] * (2**index)

        energy, bad_states = InputChecker.run_checker(f'./{unicorn_files_path}/', input_dec)
        if energy == 0:
            good_answers_overall += occs
    return good_answers_overall

def get_wrong_var_counts(bqm, samples, unicorn_files_path, input_variables):
    wrong_var_counts = []
    for (index, sample) in enumerate(samples):
        occs = samples.record[index][2]
        input_dec = 0
        
        for (index, input_variable) in enumerate(input_variables):
            input_dec += sample[input_variable] * (2**index)
        energy, bad_states = InputChecker.run_checker(f'./{unicorn_files_path}/', input_dec)

        wrong_var_count = 0
        for var in bqm.variables:
            if InputChecker.qubits_to_fix[var] != sample[var]:
                wrong_var_count += 1
        for i in range(occs):
            wrong_var_counts.append(wrong_var_count)
    return wrong_var_counts

def get_energies(samples):
    energies = []
    for sample in samples.record:
        for i in range(sample[2]):
            energies.append(sample[1])
    return energies

In [183]:
COL_FILE_NAME = 'file_name'
COL_WORDSIZE = 'wordsize'
COL_LOGIC_VARS = 'logic_vars'
COL_QPU_NAME = 'qpu_name'
COL_CHAIN_STRENGTH = 'chain_strength'
COL_NUM_SAMPLES = 'samples'
COL_TIME = 'time'

COL_AVG_ENERGY = 'avg_energy'
COL_MEDIAN_ENERGY = 'median_energy'
COL_MIN_ENERGY = 'min_energy'
COL_MAX_ENERGY = 'max_energy'

COL_COUNT_0_ENERGY = "count_0_energy"
COL_GOOD_ANSWERS = "good_answers"

COL_AVG_WRONG_VARS = 'avg_wrong_vars'
COL_MEDIAN_WRONG_VARS = 'median_wrong_vars'
COL_MIN_WRONG_VARS = 'min_wrong_vars'
COL_MAX_WRONG_VARS = 'max_wrong_vars'

columns_data_annealer = [COL_FILE_NAME, COL_WORDSIZE, COL_LOGIC_VARS, COL_QPU_NAME, 
                         COL_CHAIN_STRENGTH, COL_NUM_SAMPLES, COL_TIME, COL_AVG_ENERGY, 
                         COL_MEDIAN_ENERGY, COL_MIN_ENERGY, COL_MAX_ENERGY, COL_COUNT_0_ENERGY, 
                         COL_GOOD_ANSWERS, COL_AVG_WRONG_VARS, COL_MEDIAN_WRONG_VARS, 
                         COL_MIN_WRONG_VARS, COL_MAX_WRONG_VARS]

data_annealer = pd.DataFrame(columns=columns_data_annealer)

def add_row_data_annealer(filename, wordsize, logic_vars, qpu_name, chain_strength, num_samples, 
                          time, energies, count_0_energy, good_answers, wrong_vars):
    row = {
        COL_FILE_NAME: filename,
        COL_WORDSIZE: wordsize,
        COL_LOGIC_VARS: logic_vars,
        COL_QPU_NAME: qpu_name,
        COL_CHAIN_STRENGTH: chain_strength,
        COL_NUM_SAMPLES: num_samples,
        COL_TIME: time,
        COL_AVG_ENERGY: round(sum(energies)/len(energies),2),
        COL_MEDIAN_ENERGY: statistics.median(energies),
        COL_MIN_ENERGY: min(energies),
        COL_MAX_ENERGY: max(energies),
        COL_COUNT_0_ENERGY: count_0_energy,
        COL_GOOD_ANSWERS: good_answers,
        COL_AVG_WRONG_VARS: round(sum(wrong_vars)/len(wrong_vars),2),
        COL_MEDIAN_WRONG_VARS: statistics.median(wrong_vars),
        COL_MIN_WRONG_VARS: min(wrong_vars),
        COL_MAX_WRONG_VARS: max(wrong_vars)
    }
    return data_annealer.append(row, ignore_index=True)

In [186]:
def get_qpu_statistics(filename, wordsize, bqm, qpu_name, unicorn_files_path, input_variables):
    global data_annealer
    logic_vars = len(bqm.adj.keys())
    for chain_strength in chain_strengths_to_try:
        for num_samples in samples_numbers_to_try:
            print(chain_strength, num_samples)
            t0 = time.perf_counter()
            samples = sample_qpu(bqm, qpu_name, num_samples, chain_strength)
            t1 = time.perf_counter()
            sampling_time = round(t1-t0, 2)
            energies = get_energies(samples)
            count_0_energy = get_energy_0_count(samples)
            good_answers = get_good_answers_overall(samples, unicorn_files_path, input_variables)
            wrong_vars = get_wrong_var_counts(bqm, samples, unicorn_files_path, input_variables)
            data_annealer = add_row_data_annealer(filename, wordsize, logic_vars, qpu_name, chain_strength, 
                                                  num_samples, sampling_time, energies, count_0_energy, 
                                                  good_answers, wrong_vars)

In [229]:
get_qpu_statistics('u.c', 32, bqm_32, ADVANTAGE, '32_u', [9376, 9377, 9378, 9379, 9380, 9381, 9382, 9383])

In [176]:
get_qpu_statistics('u.c', 32, bqm_32, DWAVE_2000, '32_u', [9376, 9377, 9378, 9379, 9380, 9381, 9382, 9383])

In [188]:
data_annealer

,file_name,wordsize,logic_vars,qpu_name,chain_strength,samples,time,avg_energy,median_energy,min_energy,max_energy,count_0_energy,good_answers,avg_wrong_vars,median_wrong_vars,min_wrong_vars,max_wrong_vars
0,u.c,32,348,Advantage_system4.1,1,1000,13.78,21.81,22.0,4.0,42.0,0,872,71.15,70.0,7,131
1,u.c,32,348,Advantage_system4.1,1,1000,14.09,21.42,22.0,4.0,40.0,0,907,68.02,67.0,8,149
2,u.c,32,348,Advantage_system4.1,1,5000,20.55,21.15,20.0,4.0,46.0,0,4276,70.64,71.0,7,146
3,u.c,32,348,Advantage_system4.1,1,7000,19.90,20.40,20.0,4.0,46.0,0,6479,65.04,64.0,7,162
4,u.c,32,348,Advantage_system4.1,1,10000,28.17,20.96,20.0,4.0,46.0,0,8389,70.72,71.0,7,150
5,u.c,32,348,Advantage_system4.1,2,1000,17.77,18.77,18.0,2.0,34.0,0,962,72.51,72.0,5,160
6,u.c,32,348,Advantage_system4.1,2,5000,24.40,18.03,18.0,2.0,44.0,0,4895,66.09,65.0,2,166
7,u.c,32,348,Advantage_system4.1,2,7000,18.22,19.98,20.0,2.0,50.0,0,6875,70.68,70.0,4,182
8,u.c,32,348,Advantage_system4.1,2,10000,23.83,20.76,20.0,2.0,46.0,0,9675,71.33,70.5,3,173


In [ ]:
get_qpu_statistics('u.c', 64, bqm_64, DWAVE_2000, '64_u', [74187, 74188, 74189, 74190, 74191, 74192, 74193, 74194])

# Advantage

## 32-bit, 7k reads, 1.5 chain

In [219]:
qpu = EmbeddingComposite(DWaveSampler(solver={"name": ADVANTAGE}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_32, num_reads=7000, chain_strength=1.5)

In [221]:
result.first.energy

0.0

In [223]:
len(result.lowest())

12

## 32-bit, 7k reads, 1 chain

In [224]:
qpu = EmbeddingComposite(DWaveSampler(solver={"name": ADVANTAGE}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_32, num_reads=7000, chain_strength=1)

In [225]:
result.first.energy

6.0

## 32-bit, 10k reads, 1.5 chain

In [226]:
qpu = EmbeddingComposite(DWaveSampler(solver={"name": ADVANTAGE}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_32, num_reads=10000, chain_strength=1.5)

In [227]:
result.first.energy

0.0

In [228]:
len(result.lowest())

9

## 64-bit, 7k reads, 1.5 chain

In [236]:
t0 = time.perf_counter()
qpu = EmbeddingComposite(DWaveSampler(solver={"name": ADVANTAGE}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_64, num_reads=7000, chain_strength=1.5)
t1 = time.perf_counter()

In [237]:
print(round(t1-t0,2), result.first.energy, len(result.lowest()))

16.03 0.0 1


## 64-bit, 10k reads, 1.5 chain

In [238]:
t0 = time.perf_counter()
qpu = EmbeddingComposite(DWaveSampler(solver={"name": ADVANTAGE}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_64, num_reads=10000, chain_strength=1.5)
t1 = time.perf_counter()
print(round(t1-t0,2), result.first.energy, len(result.lowest()))

16.41 0.0 1


# Chimera

In [239]:
t0 = time.perf_counter()
qpu = EmbeddingComposite(DWaveSampler(solver={"name": DWAVE_2000}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_32, num_reads=7000, chain_strength=1.5)
t1 = time.perf_counter()
print(round(t1-t0,2), result.first.energy, len(result.lowest()))

14.12 2.0 1


In [240]:
t0 = time.perf_counter()
qpu = EmbeddingComposite(DWaveSampler(solver={"name": DWAVE_2000}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_32, num_reads=7000, chain_strength=2)
t1 = time.perf_counter()
print(round(t1-t0,2), result.first.energy, len(result.lowest()))

22.82 2.0 5


In [243]:
t0 = time.perf_counter()
qpu = EmbeddingComposite(DWaveSampler(solver={"name": DWAVE_2000}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_32, num_reads=7000, chain_strength=2.5)
t1 = time.perf_counter()
print(round(t1-t0,2), result.first.energy, len(result.lowest()))

25.18 4.0 5


In [244]:
t0 = time.perf_counter()
qpu = EmbeddingComposite(DWaveSampler(solver={"name": DWAVE_2000}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_32, num_reads=10000, chain_strength=1.5)
t1 = time.perf_counter()
print(round(t1-t0,2), result.first.energy, len(result.lowest()))

25.54 2.0 5


In [245]:
t0 = time.perf_counter()
qpu = EmbeddingComposite(DWaveSampler(solver={"name": DWAVE_2000}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm_64, num_reads=10000, chain_strength=1.5)
t1 = time.perf_counter()
print(round(t1-t0,2), result.first.energy, len(result.lowest()))

18.64 2.0 1
